#Assume \tau <= K, where \tau is lifetime random variable and K is Contract termination.
#Underling stock S_t = (r - l) S_t dt + \sigma dB_t, S_0 = 1.   Here r is interest rate, l is risk charge, and sigma is volatility of stock.
#Option payoff E_x[V^g] &= E[\exp^{-r\tau} \times \int_0^\infty Max{\exp{g\tau} - S_\tau, 0}d \phi_t]
<!-- #                       &= \int_0^K \lambda exp^{-\lambda t} \exp{-rt} \int_0^\infty Max{\exp{g\tau} - S_\tau, 0}d \phi_t dt -->
<!-- #                       here \lambda is force of death mortality, g is rising-floor rate, and \phi_t is price density function. -->

#Target when \sigma = 0.20, r = 0.06, \lambda = 1/35, l = 0.0125, g = 0
#Compare Titanic Option E_x[V^g] = 2.85%

In [4]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as spi
import sympy as sp

1. 调整T = 40，M = 100， 0.028851222661773618
1. 调整T = 40，M = 100， 0.0282524351119871
1. 调整T = 40，M = 100， 0.028337673292643025
1. 调整T = 40，M = 100， 0.02852353720652563
1. 调整T = 40，M = 100， 0.028716563920467068

In [ ]:
# 参数设置
S0 = 1      # 初始资产价格
T = 40.0    # 总时长（例如1年）
r = 0.06    # 风险无关利率
sigma = 0.2 # 波动率
dt = 0.01   # 时间步长
N = 500     # 总步数
g = 0
lamda = 1/35
M = 100  # 例如，进行100次积分
MC_number = 2   # 将5次MC的结果进行平均，更稳定

In [5]:
# 定义股价模拟函数
def stock_SDE(S0, r, sigma, dt, N, l):
    S = np.zeros(N)
    S[0] = S0
    for t in range(1, N):
        Z = np.random.normal(0, 1)
        S[t] = S[t-1] * np.exp((r - l - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
    return S[N-1]

# 定义期权收益函数
def option_payoff(g, t, S0, r, sigma, dt, N, l):
    return max(np.exp(g * t) - stock_SDE(S0, r, sigma, dt, N, l), 0)

# 定义单次积分函数
def single_V(t, l):
    return lamda * np.exp(-lamda * t) * np.exp(-r * t) * option_payoff(g, t, S0, r, sigma, dt, N, l)

# 定义多次积分值的均值计算函数
def average_int_death_payoff(l, M):
    results = []
    for _ in range(M):
        result, _ = spi.quad(single_V, 0, T, args=(l,))
        results.append(result)
    print("积分均值：",np.mean(results))
    return np.mean(results)

# 定义费用流函数
def fee_flows(l):
    return l / (lamda + l)

In [6]:

optimal_l = None
min_diff = float('inf')

j = 0

for l_candidate in np.linspace(0, 0.001, 2):  # 尝试不同的 l 值
    average_MC = []

    for _ in range(MC_number):
        average_payoff = average_int_death_payoff(l_candidate, M)
        average_MC.append(average_payoff)

    average_MC_payoff = np.mean(average_MC)
    print("回合",j,", average_MC:", average_MC_payoff)
    fee_flow = fee_flows(l_candidate)
    print("回合",j,", fee_flows:", fee_flow)

    diff = abs(average_MC_payoff - fee_flow)
    if diff < min_diff:
        min_diff = diff
        print("回合",j,", diff:",diff)
        optimal_l = l_candidate
        print("回合",j,", optimal_l:",optimal_l)
    j  = j+1
print("Optimal l:", optimal_l)

/tmp/ipykernel_8690/1270861124.py:32: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result, _ = spi.quad(single_V, 0, T, args=(l,))
/tmp/ipykernel_8690/1270861124.py:32: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  result, _ = spi.quad(single_V, 0, T, args=(l,))


积分均值： 0.02298795263922697
积分均值： 0.023588030329248815
回合 0 , average_MC: 0.023287991484237894
回合 0 , fee_flows: 0.0
回合 0 , diff: 0.023287991484237894
回合 0 , optimal_l: 0.0
积分均值： 0.023553186696881258
积分均值： 0.023792151209928468
回合 1 , average_MC: 0.023672668953404863
回合 1 , fee_flows: 0.033816425120772944
回合 1 , diff: 0.010143756167368081
回合 1 , optimal_l: 0.001
Optimal l: 0.001
